In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
pd.set_option("display.max.columns", None)


In [ ]:
results = pd.DataFrame(columns=['count','ctime','wtime','cerr','werr'])
WORK="NOPLOOP"
TYPE="LOCAL_WORK"
CPU0=3
CPU1=17
MACHINE="don"
DATE="14-11-22:15:25:37"


for COUNT in [100, 200, 400, 800, 1600, 3200, 6400, 12800, 25600, 51200]:
    FILE="data/"+MACHINE+"/"+DATE+"/bm."+WORK+"_"+TYPE+"_"+str(COUNT)+"_"+str(CPU0)+"_"+str(CPU1)+".times"
    print(FILE)
    data=pd.read_csv(FILE)
    count=data['count'][0]
    cpu0=data['cpu0'][0]
    cpu1=data['cpu1'][0]
    data['pctime']=data['ctime']/count
    data['pwtime']=data['wtime']/count
    minCtime=data['pctime'].min()
    maxCtime=data['pctime'].max()
    meanCtime=data['pctime'].mean()
    errCtime=[meanCtime - minCtime, maxCtime - meanCtime]
    minWtime=data['pwtime'].min()
    maxWtime=data['pwtime'].max()
    meanWtime=data['pwtime'].mean()
    errWtime=meanWtime - minWtime, maxWtime - meanWtime
    newres = pd.DataFrame([{'count':count, 'ctime':meanCtime, 'wtime':meanWtime, 'cerr':errCtime, 'werr':errWtime}])
    results = pd.concat([results, newres], axis=0,ignore_index=True)
results
plt.figure()
fig, ax = plt.subplots()
ax.set_ylim([0,36000])
ax.plot(results["count"],results["ctime"], label=WORK+"_"+TYPE+"_CTIME",marker='x')
ax.plot(results["count"],results["wtime"], label=WORK+"_"+TYPE+"_WTIME",marker='o')
plt.show()

In [ ]:
FILE="data/"+MACHINE+"/"+DATE+"/bm."+WORK+"_"+TYPE+"_"+str(COUNT)+"_"+str(CPU0)+"_"+str(CPU1)+".times"
print(FILE)

In [ ]:
data=pd.read_csv(FILE)

In [ ]:
data

In [ ]:
count=data['count'][0]
cpu0=data['cpu0'][0]
cpu1=data['cpu1'][0]
data['pctime']=data['ctime']/count
data['pwtime']=data['wtime']/count
minCtime=data['pctime'].min()
maxCtime=data['pctime'].max()
meanCtime=data['pctime'].mean()
errCtime=[meanCtime - minCtime, maxCtime - meanCtime]
minWtime=data['pwtime'].min()
maxWtime=data['pwtime'].max()
meanWtime=data['pwtime'].mean()
errWtime=[meanWtime - minWtime, maxWtime - meanWtime]
newres = pd.DataFrame([{'count':count, 'ctime':meanCtime, 'wtime':meanWtime, 'cerr':errCtime, 'werr':errWtime}])
results = pd.concat([results, newres], axis=0,ignore_index=True)
#results['ctime'][1] = meanCtime
#results['wtime'][1] = meanWtime
#results['cerr'][1] = errCtime
#results['werr'][1] = errWtime

In [ ]:
data

In [ ]:
errCtime,errWtime

In [ ]:
results